In [ ]:
# use tensorflow version = 1.x works on google colab
%tensorflow_version 1.x

import keras
from keras.models import  Model, load_model
from keras.layers import *
import keras.backend as K
import tensorflow as tf
import numpy as np
import os
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

In [ ]:
# using CuDNNLSTM we need GPU. 
# ensure you are using GPU
def check_GPU():
    if tf.test.is_gpu_available()==False:
        print('No GPU available. You need a GPU to use Cuda!')
        exit()
    else:
        print('GPU found!')
	
# function used to define NN geometry
# this was usefull for the gridsearch on geometry:
# inputs are:
## layers: list of 0s and 1s of shape (NN depth -1) [int]
### 0: normal LSTM
### 1: Bidirectional(LSTM)
## neurons:  list of number of neurnons of shape (NN depth) [int]
### neurons[-1] must be = 2 for this problem
def NN_LSTMgen(layers=[0,1, 0, 0], neurons=[1,32,16,8, 1]):
    # use Keras Input function: init input for NN
    main_input = Input(shape=(None, 1), name='main_input')
    # if layer == 0 --> 1st is a LSTM, else put a Bidirectional(LSTM)
    if layers[0]==0:
        x = CuDNNLSTM(neurons[0], return_sequences=True)(main_input)
    elif layers[0]==1:
        x = Bidirectional(CuDNNLSTM(neurons[0], return_sequences=True))(main_input)
    else:
        print('Wrong first layer! Must start with LSTM o Bidirectional(LSTM)')     
        exit()   
    # for each layer, stack as done before LSTM and Bidirectional(LSTM) 
    for i in range(1, np.shape(neurons)[0] - 1):
        if layers[i]==0:
            x = CuDNNLSTM(neurons[i], return_sequences=True)(x)    
        elif layers[i]==1:
            x = Bidirectional(CuDNNLSTM(neurons[i], return_sequences=True))(x)
        else:
            print('Unknown layer! Must be a LSTM [0] o Bidirectional(LSTM) [1]')   
    # last layer must be a Dense (for classification with softmax)     
    main_output = Dense(neurons[-1], activation='softmax')(x)
    # return model
    return Model(inputs=[main_input], outputs=[main_output])

# train function --> usefull for grid search
# input: number of epochs, batch_size and model_path (save path for the trained model)
def trainNN(epochs, batch=1024, pathm='model/'):
    # define Early stopping criterion
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
    # fit the model with model.fit() keras function
    history=model.fit(X_train, y_train, validation_data=(X_val, y_val),  epochs = epochs, batch_size = batch, verbose = 1, callbacks=[es])
    # save trained model
    model.save(pathm + 'model_final.h5')
    return history, model



In [ ]:
# define data path
data_path = 'data_no_noise/'
# load X_train, X_val, y_train, y_val
X_train = np.load(data_path + 'X_train.npy')
X_val   = np.load(data_path + 'X_val.npy')
y_train = np.load(data_path + 'y_train.npy')
y_val   = np.load(data_path + 'y_val.npy')

# check GPU is working and is a CUDA GPU
GPU = check_GPU()

In [ ]:
# create keras model
model = NN_LSTMgen([0, 1, 0, 0, 0], [4 ,8 ,16, 8, 4, 2])

# init optimizer
opt = keras.optimizers.Adamax(lr = 1e-3)

# compile the model
model.compile(loss='categorical_crossentroy', optimizer=opt, metrics=['acc'])

# fit the model
history_no_noise,  model_no_noise = trainNN(500, pathm='model_no_noise/')

In [ ]:
# load trained model
model = load_model(data_path + 'model_final.h5')

# define new data path
data_path = 'data_noise/'
# load X_train, X_val, y_train, y_val
X_train = np.load(data_path + 'X_train.npy')
X_val   = np.load(data_path + 'X_val.npy')
y_train = np.load(data_path + 'y_train.npy')
y_val   = np.load(data_path + 'y_val.npy')

# check GPU is working and is a CUDA GPU
GPU = check_GPU()

In [ ]:
# compile trained model and fit 
model.compile(loss='categorical_crossentroy', optimizer=opt, metrics=['acc'])
history,  model = trainNN(2500, pathm='model_no_noise/')